In [ ]:
#pip install snowflake-connector-python sqlalchemy pandas matplotlib ta yfinance pymysql snowflake-sqlalchemy python-dotenv cryptography simplejson pandas_datareader  

In [ ]:
#pip install pyqt5  plotly 

In [ ]:
#pip install kaleido

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from datetime import datetime,timedelta
import requests
import ta

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import base64
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from sqlalchemy import text

load_dotenv()

with open(os.getenv("PRIVATE_KEY_PATH"), "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )

private_key_pkcs8 = base64.b64encode(
    private_key.private_bytes(
        encoding=serialization.Encoding.DER,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
).decode("utf-8")  # ✅ base64 string, not bytes!

engine = create_engine(URL(
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    user=os.getenv("SNOWFLAKE_USER"),
    private_key=private_key_pkcs8,
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    role=os.getenv("SNOWFLAKE_ROLE")
))

with engine.connect() as conn:
    result = conn.execute(text("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_TIMESTAMP();"))
    for row in result:
        print("✅ Snowflake connected:", row)


✅ Snowflake connected: ('CRYPTO_USER', 'CRYPTO_ROLE', datetime.datetime(2025, 4, 3, 1, 11, 31, 171000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>))


In [ ]:
from datetime import datetime

def convert_to_date(date_string, date_format="%Y-%m-%d"):
    """
    Converts a date string to a datetime object.
    
    Args:
        date_string (str): The date as a string.
        date_format (str): The format of the date string (default is '%Y-%m-%d').
    
    Returns:
        datetime: A datetime object representing the date, or None if conversion fails.
    """
    try:
        return datetime.strptime(date_string, date_format)
    except ValueError:
        print(f"Error: '{date_string}' does not match the format '{date_format}'")
        return None



In [ ]:
import time

def fetch_btc_from_coinbase(granularity=86400, limit_days=300):
    """
    Fetch historical daily candles from Coinbase (granularity = 86400 seconds = 1 day).
    Coinbase limits each call to 300 candles, so this fetches as many chunks as needed.
    """
    url = "https://api.exchange.coinbase.com/products/BTC-USD/candles"
    all_data = []
    end_time = pd.Timestamp.utcnow()

    print("Fetching BTC-USD candles from Coinbase...")

    for _ in range((limit_days // 300) + 1):
        start_time = end_time - pd.Timedelta(days=300)

        params = {
            "start": start_time.isoformat(),
            "end": end_time.isoformat(),
            "granularity": granularity
        }

        r = requests.get(url, params=params)
        r.raise_for_status()
        chunk = r.json()

        all_data.extend(chunk)
        end_time = start_time
        time.sleep(1)  # avoid rate limit

    # Convert to DataFrame
    df = pd.DataFrame(all_data, columns=["timestamp", "low", "high", "open", "close", "volume"])
    df["Date"] = pd.to_datetime(df["timestamp"], unit="s")
    df = df[["Date", "open", "high", "low", "close", "volume"]]
    df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
    df.sort_values("Date", inplace=True)

    print(f"Fetched {len(df)} daily rows of BTC-USD data.")
    return df.reset_index(drop=True)


In [ ]:
btc_df = fetch_btc_from_coinbase(limit_days=4000) 
btc_df.head()


In [ ]:
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS CRYPTODB.PUBLIC.btcusd"))

In [ ]:
from sqlalchemy.types import String, Float, DateTime

btc_df.to_sql(
    'btcusd',
    con=engine,
    if_exists='replace',
    index=False,
    dtype={
        "Date": DateTime(),
        "Open": Float(),
        "High": Float(),
        "Low": Float(),
        "Close": Float(),
        "Volume": Float(),  # or String() if some values are missing
    }
)


In [ ]:
from sqlalchemy import text
from sqlalchemy.types import Integer, String, Float, DateTime

# Fetch data from Fear & Greed Index API
url = 'https://api.alternative.me/fng/?limit=0'
response = requests.get(url)
response.raise_for_status()
data = response.json()['data']

# Convert to DataFrame
df = pd.DataFrame(data)

# Convert 'timestamp' to UTC and then remove timezone info
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)
df['timestamp'] = df['timestamp'].dt.tz_localize(None)



df['value'] = pd.to_numeric(df['value'])


# Drop unneeded columns
df.drop(columns=['time_until_update','value_classification'], inplace=True)

with engine.connect() as conn:
    conn.execute(text('DROP TABLE IF EXISTS feargreedindex'))






In [ ]:
df.to_sql(
    'feargreedindex',
    con=engine,
    if_exists='replace',
    index=False,
    dtype={
        "value": Float(),
        "timestamp": DateTime()
    }   
    
)

print("✅ FearGreedIndex table successfully written to Snowflake.")

In [6]:
query = """
SELECT "Date", "Open", "High", "Low", "Close", "Volume" , cast(fgi.value AS Integer) as Fgi
FROM CRYPTODB.PUBLIC.BTCUSD btc
LEFT JOIN CRYPTODB.PUBLIC.FEARGREEDINDEX fgi
ON btc."Date" = fgi.timestamp
Order by 1 
"""

data = pd.read_sql(query,con=engine)
data.head()

,Date,Open,High,Low,Close,Volume,fgi
0,2015-07-20,277.98,280.00,277.37,280.00,782.883420,NaN
1,2015-07-21,279.96,281.27,276.85,277.32,4943.559434,NaN
2,2015-07-22,277.33,278.54,275.01,277.89,4687.909383,NaN
3,2015-07-23,277.96,279.75,276.28,277.39,5306.919575,NaN
4,2015-07-24,277.23,291.52,276.43,289.12,7362.469083,NaN


In [7]:
# Calculate the average of the FGI column, excluding NaN values
fg_average = data['fgi'].mean()

# Fill NaN values in the FGI column with the calculated average
data['fgi'].fillna(fg_average, inplace=True)

C:\Users\edufs\AppData\Local\Temp\ipykernel_15640\3904962655.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['fgi'].fillna(fg_average, inplace=True)


In [8]:
#https://www.investopedia.com/terms/m/macd.asp
#macd_object = ta.trend.MACD(data['close'])
macd_object = ta.trend.MACD(data['Close'], window_slow = 26, window_fast = 12, window_sign = 9, fillna= True)
data['MACD'] = macd_object.macd()
data['MACD_Signal'] = macd_object.macd_signal()
data['MACD_Diff'] = macd_object.macd_diff()
#data.tail()

In [9]:
#https://www.investopedia.com/terms/m/mfi.asp
mfi_indicator = ta.volume.MFIIndicator(high=data['High'], low=data['Low'], close=data['Close'], volume=data['Volume'], window=14)
data['mfi'] = mfi_indicator.money_flow_index()

In [10]:
# RSI
rsi = ta.momentum.rsi(data['Close'], window=14, fillna=True)
data['RSI'] = rsi

In [11]:
# EMA
ema_short = ta.trend.EMAIndicator(data['Close'], window=12, fillna=True)
ema_long = ta.trend.EMAIndicator(data['Close'], window=26, fillna=True)
data['EMA_Short'] = ema_short.ema_indicator()
data['EMA_Long'] = ema_long.ema_indicator()

In [12]:
# Bollinger Bands
bollinger = ta.volatility.BollingerBands(data['Close'], window=20, window_dev=2, fillna=True)
data['Bollinger_Upper'] = bollinger.bollinger_hband()
data['Bollinger_Lower'] = bollinger.bollinger_lband()
data['Bollinger_Middle'] = bollinger.bollinger_mavg()

In [13]:
# Stochastic Oscillator
stoch = ta.momentum.StochasticOscillator(data['High'], data['Low'], data['Close'], window=14, smooth_window=3, fillna=True)
data['Stochastic'] = stoch.stoch()
data['Stochastic_Signal'] = stoch.stoch_signal()

In [14]:
# ATR
atr = ta.volatility.AverageTrueRange(data['High'], data['Low'], data['Close'], window=14, fillna=True)
data['ATR'] = atr.average_true_range()

In [15]:
# Parabolic SAR
sar = ta.trend.PSARIndicator(data['High'], data['Low'], data['Close'], fillna=True)
data['SAR'] = sar.psar()

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the data (assuming 'data' is already pre-processed with features)
data['Date'] = pd.to_datetime(data['Date'], utc=True)
data['Target'] = data['Close'].shift(-1)
data['Return'] = data['Close'].pct_change().fillna(0)
data['Volatility'] = data['Return'].rolling(window=7).std().fillna(0)
data['Volatility_x_Return'] = data['Volatility'] * data['Return']

data = data.dropna(subset=['Target'])

# Select features (excluding 'Date')
features = [
     'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Middle','fgi', 'Close','Return','Volatility','Volatility_x_Return'
]

X = data[features]
y = data['Target']

In [17]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [18]:
# Standardize the features
scaler = StandardScaler()
scaler.fit(X_train)  # Or your main feature training data

# Then scale:
x_train_scaled = scaler.transform(X_train)
x_test_scaled = scaler.transform(X_test)


# Train the model
model = LinearRegression()
model.fit(x_train_scaled, y_train)

# Make predictions on the test data
y_pred = model.predict(x_test_scaled)

In [19]:



# Make predictions for the next closing price
# Use the last row from X_test (or any new data)
latest_data = X_test.iloc[-1:]  # Keep it as a DataFrame (not numpy array)

# Apply the same scaling to the latest data
latest_data_scaled = scaler.transform(latest_data)  # The feature names are kept

# Make prediction for the next closing price
next_close_pred = model.predict(latest_data_scaled)
print(f"Predicted next closing price: {next_close_pred[0]}")

Predicted next closing price: 82151.53401667508


In [20]:
# Extract the coefficients and the intercept
coefficients = model.coef_
intercept = model.intercept_

# Create a dictionary mapping feature names to their corresponding coefficients
feature_names = X.columns
equation = f"Predicted next closing price = {intercept:.2f}"

# Add each feature and its coefficient to the equation
for feature, coef in zip(feature_names, coefficients):
    equation += f" + ({coef:.2f}) * {feature}"

# Display the equation
print("Final equation for predicting the next closing price:")
print(equation)


Final equation for predicting the next closing price:
Predicted next closing price = 14668.01 + (338.79) * Bollinger_Upper + (-265.68) * Bollinger_Lower + (74.04) * Bollinger_Middle + (42.27) * fgi + (16102.62) * Close + (34.25) * Return + (-14.92) * Volatility + (-56.67) * Volatility_x_Return


In [21]:


import plotly.graph_objects as go

# Create a DataFrame to store the actual vs predicted values
results = pd.DataFrame({
    'Date': X_test.index,
    'Actual': y_test,
    'Predicted': y_pred
})

# Make sure Date is datetime (Plotly likes it)
results['Date'] = data.loc[X_test.index, 'Date'].values

# Build interactive plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Closing Price',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Predicted'],
    mode='lines',
    name='Predicted Closing Price',
    line=dict(color='red', dash='dash')
))

# Layout styling
fig.update_layout(
    title='Actual vs Predicted Closing Prices',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    legend=dict(x=0, y=1.1, orientation='h'),
    autosize=True,
    margin=dict(l=40, r=40, t=80, b=40),
    height=500,
)

# Show the plot
fig.show()



In [22]:
# Evaluate the model
r2 = r2_score(y_test, y_pred)

print(f"R-squared: {r2}")

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

#how far off my predictions are from actual values
print(f"MAE: {mae:.2f}")

print(f"RMSE: {rmse:.2f}")

R-squared: 0.9952795950302875
MAE: 1079.71
RMSE: 1653.26


C:\Users\edufs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



In [23]:
# Calculate absolute error and rolling MAE
results['Absolute_Error'] = abs(results['Actual'] - results['Predicted'])
results['MAE_Rolling'] = results['Absolute_Error'].rolling(window=7).mean()
# Calculate squared error and rolling RMSE
results['Squared_Error'] = (results['Actual'] - results['Predicted']) ** 2
results['RMSE_Rolling'] = results['Squared_Error'].rolling(window=7).mean().pow(0.5)

# Plot Rolling MAE and RMSE on the same chart
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['RMSE_Rolling'],
    mode='lines',
    name='Rolling RMSE',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['MAE_Rolling'],
    mode='lines',
    name='Rolling MAE',
    line=dict(color='blue')
))

fig.update_layout(
    title='7-Day Rolling MAE and RMSE Over Time',
    xaxis_title='Date',
    yaxis_title='Error (USD)',
    template='plotly_white'
)

fig.show()
fig.write_image("assets/Rolling_MAE_RMS.png")


In [25]:
import statsmodels.api as sm
import plotly.express as px

# Store original column names
original_columns = X_train.columns

# Standardize the features (X_train_scaled) for fitting the model
X_with_intercept = sm.add_constant(x_train_scaled)  # Add intercept term to the features
y_train_array = y_train.values  # Convert to numpy array for Statsmodels compatibility

# Fit the model using Statsmodels
model_sm = sm.OLS(y_train_array, X_with_intercept)  # Ordinary Least Squares Regression
results = model_sm.fit()  # Fit the model

# Display the summary, which includes the p-values for each feature
summary = results.summary()

# Map p-values to the original feature names
p_values = results.pvalues[1:]  # Exclude the intercept (constant)
features_with_p_values = dict(zip(original_columns, p_values))

# Print the p-values for each feature and check significance
print("Feature p-values (human-readable labels):")
for feature, p_value in features_with_p_values.items():
    significance = "Significant" if p_value < 0.05 else "Not Significant"
    print(f"{feature}: {p_value:.4f} ({significance})")

# Optionally, print the full summary
print("\nFull summary with p-values:")
print(summary)


# Convert to DataFrame for plotting
pval_df = pd.DataFrame({
    'Feature': list(features_with_p_values.keys()),
    'P_Value': list(features_with_p_values.values())
})

# Sort by P-value
pval_df = pval_df.sort_values('P_Value')

# Plot
fig = px.bar(
    pval_df,
    x='Feature',
    y='P_Value',
    color=pval_df['P_Value'] < 0.05,
    color_discrete_map={True: 'green', False: 'red'},
    title="Feature Significance (P-Values)",
    labels={'P_Value': 'P-Value', 'Feature': 'Feature'},
    template='plotly_white'
)

fig.update_layout(
    yaxis_range=[0, 1],
    showlegend=False
)

fig.add_hline(y=0.05, line_dash="dash", line_color="black")

fig.show()





Feature p-values (human-readable labels):
Bollinger_Upper: 0.0031 (Significant)
Bollinger_Lower: 0.0112 (Significant)
Bollinger_Middle: 0.1564 (Not Significant)
fgi: 0.0245 (Significant)
Close: 0.0000 (Significant)
Return: 0.3159 (Not Significant)
Volatility: 0.3904 (Not Significant)
Volatility_x_Return: 0.0902 (Not Significant)

Full summary with p-values:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.527e+05
Date:                Thu, 03 Apr 2025   Prob (F-statistic):               0.00
Time:                        04:15:09   Log-Likelihood:                -23106.
No. Observations:                2836   AIC:                         4.623e+04
Df Residuals:                    2828   BIC:                         4.628e+04
Df Model

In [26]:
import plotly.figure_factory as ff

# Select only numeric columns
numeric_df = data[features + ['Target']].copy()

# Calculate correlation matrix
corr_matrix = numeric_df.corr().round(2)

# Create heatmap
fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns.tolist(),
    y=corr_matrix.index.tolist(),
    colorscale='Blues',
    showscale=True,
    annotation_text=corr_matrix.values.round(2)
)

fig.update_layout(
    title='📈 Feature Correlation Heatmap (Pearson)',
    template='plotly_white',
    height=600
)

fig.show()

fig.write_image("assets/Feature_Correlation.png")

In [28]:
import plotly.graph_objects as go

# Create a DataFrame to store the actual vs predicted values
results = pd.DataFrame({
    'Date': X_test.index,
    'Actual': y_test,
    'Predicted': y_pred
})

# Make sure Date is datetime (Plotly likes it)
results['Date'] = data.loc[X_test.index, 'Date'].values

# Build interactive plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Closing Price',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Predicted'],
    mode='lines',
    name='Predicted Closing Price',
    line=dict(color='red', dash='dash')
))

# Layout styling
fig.update_layout(
    title='Actual vs Predicted Closing Prices',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    legend=dict(x=0, y=1.1, orientation='h'),
    autosize=True,
    margin=dict(l=40, r=40, t=80, b=40),
    height=500,
)

# Show the plot
fig.show()


In [29]:
import plotly.express as px


# Assume results DataFrame contains 'Actual' and 'Predicted' columns

# Melt the DataFrame so we can compare both Actual and Predicted in a single boxplot
box_df = results.melt(
    value_vars=['Actual', 'Predicted'],
    var_name='Type',
    value_name='Price'
)

# Create boxplot
fig = px.box(
    box_df,
    x='Type',
    y='Price',
    color='Type',
    title='Distribution of Actual vs Predicted Prices',
    points='all',  # Show all data points (adds scatter to boxplot)
    template='plotly_white'
)

fig.update_layout(
    yaxis_title='BTC Price (USD)',
    xaxis_title='Type',
    showlegend=False
)

fig.show()
fig.write_image("assets/ActualvsPredicted.png")

In [30]:
import plotly.graph_objects as go

# Calculate rolling metrics if not already done
results['Absolute_Error'] = abs(results['Actual'] - results['Predicted'])
results['MAE_Rolling'] = results['Absolute_Error'].rolling(window=7).mean()
results['Squared_Error'] = (results['Actual'] - results['Predicted']) ** 2
results['RMSE_Rolling'] = results['Squared_Error'].rolling(window=7).mean() ** 0.5

# Create figure with secondary y-axis
fig = go.Figure()

# Primary y-axis: Predicted price
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Predicted'],
    name='Predicted Price',
    mode='lines',
    line=dict(color='blue')
))

# Secondary y-axis: MAE
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['MAE_Rolling'],
    name='Rolling MAE',
    mode='lines',
    yaxis='y2',
    line=dict(color='orange', dash='dash')
))

# Secondary y-axis: RMSE
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['RMSE_Rolling'],
    name='Rolling RMSE',
    mode='lines',
    yaxis='y2',
    line=dict(color='red', dash='dot')
))

# Layout
fig.update_layout(
    title='Predicted BTC Price with Rolling MAE and RMSE',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Predicted Price (USD)', side='left'),
    yaxis2=dict(
        title='Error (USD)',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    legend=dict(x=0, y=1.1, orientation='h'),
    template='plotly_white',
    height=500
)

fig.show()


In [31]:
# Calculate % error metrics (avoid division by zero just in case)
results['MAE_PCT'] = (results['Absolute_Error'] / results['Predicted'].replace(0, 1)) * 100
results['RMSE_PCT'] = (results['RMSE_Rolling'] / results['Predicted'].replace(0, 1)) * 100

MAEWorstSpike = results['MAE_PCT'].max()
RMSEWorstSpike = results['RMSE_PCT'].max()

import plotly.graph_objects as go

fig = go.Figure()

fig.add_hline(
    y=3,
    line_dash="dash",
    line_color="black",
    annotation_text="3% Error Threshold",
    annotation_position="top left"
)

fig.add_hline(
    y=results['MAE_PCT'].mean(),
    line_dash="dash",
    line_color="purple",
    annotation_text="Average MAE",
    annotation_position="top left"
)

fig.add_hline(
    y=results['RMSE_PCT'].mean(),
    line_dash="dash",
    line_color="green",
    annotation_text="Average RMSE",
    annotation_position="top left"
)

fig.add_hline(
    y=MAEWorstSpike,
    line_dash="dash",
    line_color="green",
    annotation_text="Worst MAE Spike",
    annotation_position="top left"
)

fig.add_hline(
    y=RMSEWorstSpike,
    line_dash="dash",
    line_color="green",
    annotation_text="Worst RMSE Spike",
    annotation_position="top left"
)



fig.add_trace(go.Bar(
    x=results['Date'],
    y=results['MAE_PCT'],
    name='MAE %',
    marker_color='blue',
    opacity=0.6
))

fig.add_trace(go.Bar(
    x=results['Date'],
    y=results['RMSE_PCT'],
    name='RMSE %',
    marker_color='red',
    opacity=0.6
))

fig.update_layout(
    title='Prediction Error as % of Predicted BTC Price',
    xaxis_title='Date',
    yaxis_title='Error (%)',
    barmode='overlay',  # or 'group' if you want side-by-side bars
    template='plotly_white',
    height=500
)

fig.show()
fig.write_image("assets/Prediction_Error_MAE_RMSE.png")



In [36]:
from scipy.stats import gaussian_kde

fig = go.Figure()


mae_vals = results['Absolute_Error'].dropna()
rmse_vals = results['Squared_Error'].dropna() ** 0.5

# Generate KDE for MAE
mae_kde = gaussian_kde(mae_vals)
mae_x = np.linspace(mae_vals.min(), mae_vals.max(), 200)
mae_y = mae_kde(mae_x)

# Generate KDE for RMSE
rmse_kde = gaussian_kde(rmse_vals)
rmse_x = np.linspace(rmse_vals.min(), rmse_vals.max(), 200)
rmse_y = rmse_kde(rmse_x)

# RMSE Histogram + KDE FIRST (goes behind)
fig.add_trace(go.Histogram(
    x=rmse_vals,
    name='RMSE',
    opacity=0.6,
    nbinsx=40,
    marker_color='red',
    xbins=dict(size=250)
))
fig.add_trace(go.Scatter(
    x=rmse_x,
    y=rmse_y * len(rmse_vals) * 250,
    name='RMSE KDE',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='RMSE Error Distribution with KDE',
    xaxis_title='Error Value (USD)',
    yaxis_title='Frequency',
    template='plotly_white',
    legend=dict(x=0.75, y=0.95)
)


fig.show()
fig.write_image("assets/RMSE_Error_Distribution.png")



In [37]:
# MAE Histogram + KDE only
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=mae_vals,
    name='MAE',
    opacity=0.6,
    nbinsx=40,
    marker_color='blue',
    xbins=dict(size=250)
))
fig.add_trace(go.Scatter(
    x=mae_x,
    y=mae_y * len(mae_vals) * 250,
    name='MAE KDE',
    line=dict(color='blue', width=2)
))

fig.update_layout(
    title='MAE Error Distribution with KDE',
    xaxis_title='Error Value (USD)',
    yaxis_title='Frequency',
    template='plotly_white',
    legend=dict(x=0.75, y=0.95)
)

fig.show()


In [38]:
import plotly.express as px

# Calculate residuals
#results['Residual'] = results['Actual'] - results['Predicted']

# Plot histogram with KDE-style curve
fig = px.histogram(
    results,
    x='Actual',
    nbins=50,
    title='Distribution of Actual Prices',
    marginal='violin',  # optional: adds a violin plot on the side
    opacity=0.7
)

fig.update_layout(
    xaxis_title='Actual',
    yaxis_title='Count',
    bargap=0.1,
    template='plotly_white'
)

fig.show()

In [39]:
import plotly.express as px


# Plot histogram with KDE-style curve
fig = px.histogram(
    results,
    x='Predicted',
    nbins=50,
    title='Distribution of Predicted Prices',
    marginal='violin',  # optional: adds a violin plot on the side
    opacity=0.7
)

fig.update_layout(
    xaxis_title='Predicted',
    yaxis_title='Count',
    bargap=0.1,
    template='plotly_white'
)

fig.show()

In [40]:
import plotly.express as px

# Calculate residuals
results['Residual'] = results['Actual'] - results['Predicted']

# Plot histogram with KDE-style curve
fig = px.histogram(
    results,
    x='Residual',
    nbins=50,
    title='Distribution of Residuals',
    marginal='violin',  # optional: adds a violin plot on the side
    opacity=0.7
)

fig.update_layout(
    xaxis_title='Residual (Actual - Predicted)',
    yaxis_title='Count',
    bargap=0.1,
    template='plotly_white'
)

fig.show()

fig.write_image("assets/Distribution_Residuals.png")




In [41]:
print("Actual price range:", results['Actual'].min(), "-", results['Actual'].max())
print("Predicted price range:", results['Predicted'].min(), "-", results['Predicted'].max())
print("Predicted price std dev:", results['Predicted'].std())


Actual price range: 25123.34 - 106159.26
Predicted price range: 25062.09864315907 - 105872.0134988656
Predicted price std dev: 24020.210102986664


In [42]:
import plotly.express as px

fig = px.scatter(
    results,
    x='Actual',
    y='Predicted',
    title='Actual vs Predicted Prices',
    labels={'Actual': 'Actual BTC Price', 'Predicted': 'Predicted BTC Price'},
    trendline='ols',
    template='plotly_white'
)

fig.show()
fig.write_image("assets/ActualVsPredicted.png")


In [43]:
# Align predictions back to the original index
data.loc[results.index, 'Predicted'] = results['Predicted'].values

In [44]:
data.to_sql("BTC_ModelOutput", con=engine, if_exists="replace", index=False)


3545